# AGU Harvester Notebook - Version 2

Process: Version 2 is different in the way it treats the Role queries. When there is a big list of roles to query, the python can be much slower and fail to append each item to a DataFrame, so instead this version saves the data locally as json files and later will push the data to a DataFrame. This makes it easier to stop/start my queries at any point and pick up where I left off.

First I query for paper metadata, put it into a DataFrame, then extract a list of the Role IDs for each author of the papers from the DataFrame. Then a new API query is run for each RoleID and exports the resulting metadata to a new json file saved locally in a 'json_roles' folder'. After all the roles are retrieved, I append the json data from each file to a DataFrame of affiliations.

Finally, the abstracts and the affiliations are merged together into a single data frame, and I output the records to json and tagged format.

API Note: The role queries may stall part way through, but can be stopped and continued as needed. About 100k roles may take about 3+ hours.

----------

Input:
- Meeting code and Publication/meeting info
- Local path to run this notebook

Outfiles:
- Excel files: AGU Abstracts ("papers.xlsx"), Affiliation/Roles ("roles.xlsx"), and final merged data ("results.xlsx")
- AGU Author Affiliations/Roles as individual json files in directory "json_roles"
- Json file and ADS Tagged file for ingest

In [1]:
import sys
import requests
import math
import json
import pandas as pd
import sys, os, io
import argparse
import csv
import time
import datetime
import numpy as np
from numpy import nan
import re
import unicodedata
from pyingest.serializers.classic import Tagged

# Inputs
path = '/Users/sao/Documents/Python-Projects/AGU/version2/'
meetingcode = "fm22"
pub = "AGU Fall Meeting 2022, held in Chicago, IL, 12-16 December 2022, id. "
pubdate = "12/2022"

# Outputs
json_output = meetingcode + ".json"
tagged = meetingcode + ".tag"
papers_outfile = meetingcode + "_papers.xlsx"
roles_outfile = meetingcode + "_roles.xlsx"
merged_outfile = meetingcode + "_results.xlsx"


# Get AGU Meeting Abstracts, Author Roles w/ Affiliations

In [2]:
# ABSTRACTS API REQUEST
# Check if papers file already exists, then proceed with gathering Role IDs
# else: query the API for papers and then gather Role IDs

os.chdir(path)
if os.path.exists(papers_outfile):
    paper_results = pd.read_excel(papers_outfile)
    print('Read',len(paper_results),'papers from file')
    
    dt = paper_results.explode("ChildList_Role")
    roles = [eval(l) for l in dt['ChildList_Role'].to_list()]
        # Extract role IDs from data, resulting in list of lists
        # The function 'eval' does this: 
        # Transform
        #  ["['Role/4501570', 'Role/4503884']","['Role/4191734', 'Role/4191863']"] 
        # into
        # [['Role/4501570', 'Role/4503884'],['Role/4191734', 'Role/4191863']]
        # Initially the result is not a proper list of lists, but a list of strings
        # that need to be turned into proper lists.
        # Now we need to turn the list of lists into just one big list ("flatten" the list)
    roles = [item for sublist in roles for item in sublist]
    roles = list(set(roles))
    print('Read',len(roles),'author roles from file')
else:
    # API Query for Paper data
    domain = "https://agu.confex.com/agu/meetingapi.cgi/Paper"
    AGU_API = domain[:27] + meetingcode + '/' + domain[27:]
    data = requests.get(AGU_API).json()
    print('Got',len(data),'papers from',meetingcode)

    # Extract specified metadata from results
    paper_results = pd.json_normalize(data)
    paper_results = pd.DataFrame(paper_results)
    paper_results = paper_results[[
        "Abstract",
        "Title",
        "ChildList_Role",
        "FinalPaperNumber",
        "_url",
        "Withdrawn",
        "GoodType"
    ]]

    # Drop rows where GoodType = Break (these aren't abstracts); Drop withdrawn papers
    paper_results = paper_results[paper_results.GoodType != 'Break']
    paper_results = paper_results[paper_results.Withdrawn != 'w']
    paper_results = paper_results.drop('Withdrawn',axis=1)
    
    # Save excel file of paper results
    paper_results.to_excel(papers_outfile, sheet_name='paper_results', index=False)
    print("Finalized and saved",len(paper_results),"papers to",papers_outfile)

    # Prepare roles list to query API for roles/affiliations
    # Role IDs to role_list and deduplicate
    dt = paper_results.explode("ChildList_Role")
    roles = dt['ChildList_Role'].to_list()
    roles = list(set(roles))
    roles = [item for item in roles if not(pd.isnull(item)) == True]
    print('Extracted', len(roles),'author roles to query')

# ----------------------------------------------------------------------------------------
# ROLES API REQUEST
domain = "https://agu.confex.com/agu/meetingapi.cgi/"
AGU_API = domain[:27] + meetingcode + '/' + domain[27:]

# Check if roles folder exists, else make one
if os.path.exists("json_roles"):
    os.chdir("json_roles")
else:
    os.mkdir("json_roles")
    os.chdir("json_roles")

print('Started role requests at',datetime.datetime.now(),'...\n') # Start clock on API queries
file_counter = 0
# For each roleID in the papers list ("roles"), check if a json file exists,
# else: send an API request for that roleID, and make a new json file, labeled by Role ID number
for ident in roles:
    file = "{}.json"
    
    if os.path.exists(file.format(ident[5:])):
        file_counter += 1
    else:
        AGU_AUTHORS = AGU_API + str(ident)
        try:
            data = requests.get(AGU_AUTHORS).json()
            new_row = {
            "RoleID":data["_url"],
            "Role":data["Role"],
            "PaperID":data["Parent_Entry"],
            "Position":data["Priority"],
            "FirstName":data["Person_FirstName"],
            "LastName":data["Person_LastName"],
            "MiddleName":data["Person_MiddleName"],
            "Affiliation":data["Person_Affiliation"],
            "City":data["Person_City"],
            "Country":data["Person_Country"],
            "ORCID":data["Person_ORCIDiD"]
            }
            
            with open(file.format(ident[5:]),"w") as outfile:
                json.dump(new_row, outfile)
            file_counter += 1
        # Print error message for failed API query
        except Exception as err:
            print("Harvesting failed for {0} with the following reason: {1}".format(ident, err))
print('Finished role requests at',datetime.datetime.now()) # Stop clock on API queries
print('Retrieved',file_counter,'author roles \n')


## Convert author roles - push json files to excel

In [ ]:
# Define path to roles json files
path_to_json = path + "json_roles/"
json_files = [file for file in os.listdir(path_to_json) if file.endswith('.json')]

# Define pandas Dataframe with the columns I want to get from the json
role_results = pd.DataFrame(columns=['RoleID','Role','PaperID','Position','FirstName',
                                     'LastName','MiddleName','Affiliation','City','Country','ORCID'])

# Check if file already exists, else: push the json files into a new excel "roles.xlsx"
if os.path.exists(path + roles_outfile):
    print("Author affiliation results in",roles_outfile)
else:
    # We need both the json and an index number, so use enumerate()
    for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js)) as json_file:
            json_text = json.load(json_file)

            # here you need to know the layout of your json and each json has to have the same structure
            roleID = json_text['RoleID']
            role = json_text['Role']
            paperID = json_text['PaperID']
            position = json_text['Position']
            firstname = json_text['FirstName']
            lastname = json_text['LastName']
            middlename = json_text['MiddleName']
            affiliation = json_text['Affiliation']
            city = json_text['City']
            country = json_text['Country']
            orcid = json_text['ORCID']

            # here I push a list of data into a pandas DataFrame at row given by 'index'
            role_results.loc[index] = [roleID, role, paperID, position, firstname, lastname, middlename, affiliation, city, country, orcid]                      

# Save excel file of role results
    role_results.to_excel(path + roles_outfile, sheet_name='role_results', index=False)
    print("Saved author affiliation results to", roles_outfile)
                            

## Merge authors & abstracts

In [ ]:
# Check if file already exists, else: transform the data and merge; save to new excel "results.xlsx"
if os.path.exists(path + merged_outfile):
    print("Final merged results in",merged_outfile)
    
else:
    
    # Read abstracts and role results from excel, and make DataFrames
    paper_results = pd.read_excel(path + papers_outfile, sheet_name='paper_results')
    aff_results = pd.read_excel(path + roles_outfile, sheet_name='role_results')
    papers = pd.DataFrame(paper_results)
    affs = pd.DataFrame(aff_results)

    # Method to remove HTML tags from abstracts
    def remove_tags(string):
        result = re.sub('<.*?>','',string)
        return result

    # Format Abstracts
    papers["Abstract"] = (
        papers["Abstract"].astype(str)
        .replace({'_x000D_\n\t_x000D_\n_x000D_\n':''}, regex=True)
        .replace({'_x000D_\n_x000D_\n':''}, regex=True)
        .replace({'_x000D_\n\t':''}, regex=True)
        .apply(lambda cw : remove_tags(cw))
    )

    # Format Titles
    papers["Title"] = (
        papers["Title"].apply(lambda cw : remove_tags(cw))
        .replace({'_x000D_\n':''}, regex=True)
    )

    # Format Publications
    papers["Pub"] = pub + papers["FinalPaperNumber"] + "."

    # Format links
    papers["properties.ELECTR"] = "https://agu.confex.com/agu/" + meetingcode + "/meetingapp.cgi/" + papers["_url"]

    # Format Authors
    # create a new column "Author" by concatenating last name, first name, and middle name (if not NA)
    affs["Author"] = affs["LastName"] + ", " + affs["FirstName"]
    affs.loc[~affs["MiddleName"].isna(), "Author"] += " " + affs["MiddleName"]

    # Format ORCiDs
    affs["ORCIDs"] = "<ORCID>" + affs["ORCID"] + "</ORCID>"

    # Format full affiliations
    affs["Aff_full"] = (
        affs["Affiliation"] + ", " + 
        affs["City"] + ", " + 
        affs["Country"] + " " + 
        affs["ORCIDs"])

    # Replace missing values with affiliations without ORCID
    affs["Aff_full"] = affs["Aff_full"].fillna(
        affs["Affiliation"] + ", " + 
        affs["City"] + ", " + 
        affs["Country"])

    # Sort author/aff list by Paper, and then Position; Sort paper list by PaperID
    affs = affs.sort_values(by=['PaperID', 'Position'])
    papers = papers.sort_values(by=['_url'])

    # Select needed columns as new dataframes
    affs_data = affs[['Author','Aff_full','PaperID','Position']]
    papers_data = papers[['Abstract','Title','Pub','_url','properties.ELECTR']]

    # Aggregate Authors and their Affiliations by PaperID
    affs_data = affs_data.replace(np.nan,'N/A')
    affs_data = affs_data.groupby(["PaperID"]).agg(Authors=("Author", "; ".join), Aff_full=("Aff_full", "; ".join))

    # Merge papers and authors/affils by PaperID
    papers_data.rename(columns = {'_url':'PaperID'}, inplace = True)
    merged = pd.merge(papers_data, affs_data, how='left', on='PaperID')

    # Save merged data to new excel "results.xlsx"
    merged.to_excel(path + merged_outfile, sheet_name='final_results', index=False)
    print("Saved",len(merged),"records to",merged_outfile)

## Output final data as json and tagged format records

In [ ]:
# Curation note: Go into final results excel file and remove 'test abstracts', etc. Fill in missing pub info.
# Then proceed with json and tagged process

# Read curated "results.xlsx" file
os.chdir(path)
data = pd.read_excel(merged_outfile, sheet_name=0)
dt = pd.DataFrame(data)

# Grab metadata from columns and create lists
auths_ls = [a if a != "nan" else "" for a in dt["Authors"].astype(str)]
# affs_ls = [a if a != "nan" else "" for a in dt["Aff_full"].astype(str)]
affs_ls = [a.split('; ') if a != "nan" else [] for a in dt["Aff_full"].astype(str)]
abs_ls = [a if a != "nan" else "" for a in dt["Abstract"].astype(str)]
titles_ls = [t if t != "nan" else "" for t in dt["Title"].astype(str)]
pubs_ls = [p if p != "nan" else "" for p in dt["Pub"].astype(str)]
ELECTR_ls = [e if e != 'nan' else "" for e in dt['properties.ELECTR'].astype(str)]
links_ls = [{"ELECTR": e} if e else "" for e in ELECTR_ls]

# Repackage data into list of records
records = []
for auths, affs, title, pub, abstract, link in zip(auths_ls, affs_ls, titles_ls, pubs_ls, abs_ls, links_ls):
    records.append({"authors":auths,
                    "affiliations":affs,
                    "pubdate":pubdate,
                    "title":title,
                    "publication":pub,
                    "abstract":abstract,
                    "properties":link,
                    "source":"ADS"})
        
# Save json file of complete records
with open(path + json_output, 'w') as outfile:
    json.dump(records, outfile)
print("Saved",len(records),"records as",json_output)

In [ ]:
# Pyingest Serializer - Transform json into tagged format
f = open(path + json_output)
json_file = json.load(f)
outputfp = open(path + tagged, 'a')
for record in json_file:
    serializer = Tagged()
    serializer.write(record, outputfp)
outputfp.close()
print("Saved records as",tagged)

# Curation note: remove the "N/A"s from the tagged file